# 🔬 STF-Mamba: HLL Temporal Flicker Analysis
## Empirical Proof-of-Concept for the HLL Hypothesis

**Goal:** Before building V8.0, we need empirical proof that the HLL (High-Temporal, Low-Spatial)
wavelet sub-band actually separates real from fake videos across manipulation methods and datasets.

**The Hypothesis:**  
> Deepfake videos have elevated and patterned HLL energy compared to real videos,  
> because face blending creates temporal inconsistencies at blend boundaries.

**If the hypothesis holds:** HLL is a valid forensic signal → proceed with V8.0.  
**If it fails:** We need a different core signal before building anything.

### Sections
1. Environment Setup  
2. Core Analysis Tools (DWT, Frame Diff)  
3. Dataset Discovery  
4. ✅ Baseline: Frame Difference (Sanity Check)  
5. ✅ HLL Energy Analysis (Primary Hypothesis)  
6. ✅ Temporal Trajectory Analysis  
7. ✅ Spatial HLL Heatmaps  
8. ✅ Cross-Dataset Validation (Celeb-DF)  
9. ✅ SBI vs Real Deepfake Comparison  
10. 📋 Summary & V8.0 Recommendations

## Section 1 — Environment Setup

In [ ]:
# Install only what Kaggle doesn't have pre-installed
import subprocess, sys

# Only install packages not on Kaggle by default
packages = ['scipy']  # numpy, cv2, matplotlib are pre-installed
for pkg in packages:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', pkg], check=False)
print("✅ Dependencies ready")


In [ ]:
import os, json, random, warnings
from pathlib import Path
from glob import glob
from typing import List, Tuple, Dict, Optional
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
from scipy import stats
warnings.filterwarnings('ignore')

# ── Reproducibility ──────────────────────────────────────────────────────────
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# ── Output directory ─────────────────────────────────────────────────────────
OUTPUT_DIR = Path('/kaggle/working/hll_analysis')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
PLOTS_DIR = OUTPUT_DIR / 'plots'
PLOTS_DIR.mkdir(exist_ok=True)

print(f"📁 Outputs → {OUTPUT_DIR}")
print(f"🐍 NumPy {np.__version__}, OpenCV {cv2.__version__}")


## Section 2 — Core Analysis Tools

Pure NumPy implementations — no PyTorch needed.  
Runs on Kaggle CPU or T4 GPU (same results).

### The 3D-DWT Explained
We decompose the video volume **(T, H, W)** using Symlet-2 wavelets along all three axes:
- **H axis (temporal):** high-pass filter → catches frame-to-frame changes
- **L axes (spatial):** low-pass filter → removes spatial detail, keeps smooth regions

**HLL = High(T) × Low(H) × Low(W)** = pure temporal flicker with no spatial frequency content.
This is exactly what face-blending artifacts look like: temporally inconsistent, spatially smooth regions.

In [ ]:
# ─── Symlet-2 (Daubechies-2) filter coefficients ────────────────────────────
# Same as wavelet_constants.py in STF-Mamba backbone
SYM2_LO = np.array([-0.12940952255092145,  0.22414386804185735,
                      0.83651630373746899,  0.48296291314469025])
SYM2_HI = np.array([-0.48296291314469025,  0.83651630373746899,
                    -0.22414386804185735, -0.12940952255092145])

def _conv1d_strided(signal: np.ndarray, filt: np.ndarray, axis: int) -> np.ndarray:
    """Apply 1D filter along `axis` with stride=2 (decimation) and reflect padding."""
    n = signal.shape[axis]
    pad = [(0, 0)] * signal.ndim
    pad[axis] = (len(filt) - 1, len(filt) - 1)
    padded = np.pad(signal, pad, mode='reflect')
    
    out_len = n // 2
    out_shape = list(signal.shape)
    out_shape[axis] = out_len
    out = np.zeros(out_shape, dtype=np.float32)
    
    for i in range(out_len):
        sl_in  = [slice(None)] * signal.ndim
        sl_out = [slice(None)] * signal.ndim
        sl_in[axis]  = slice(i * 2, i * 2 + len(filt))
        sl_out[axis] = i
        chunk = padded[tuple(sl_in)]
        out[tuple(sl_out)] = np.tensordot(chunk, filt, axes=([axis], [0]))
    return out


def compute_hll(frames_rgb: np.ndarray) -> Tuple[float, np.ndarray]:
    """
    Compute HLL sub-band from video frames.
    
    Args:
        frames_rgb: (T, H, W, 3) uint8 — RGB video frames
    Returns:
        energy: scalar — mean squared HLL coefficient (the key metric)
        hll:    (T//2, H//2, W//2) array — HLL sub-band map
    """
    # Grayscale (T, H, W) in [0, 1]
    gray = (0.299 * frames_rgb[..., 0].astype(np.float32) +
            0.587 * frames_rgb[..., 1].astype(np.float32) +
            0.114 * frames_rgb[..., 2].astype(np.float32)) / 255.0
    
    # HLL: High(T=axis0) → Low(H=axis1) → Low(W=axis2)
    h   = _conv1d_strided(gray,  SYM2_HI, axis=0)   # temporal high-pass
    hl  = _conv1d_strided(h,     SYM2_LO, axis=1)   # spatial low-pass (height)
    hll = _conv1d_strided(hl,    SYM2_LO, axis=2)   # spatial low-pass (width)
    
    energy = float(np.mean(hll ** 2))
    return energy, hll


def compute_all_subbands(frames_rgb: np.ndarray) -> Dict[str, float]:
    """Compute all 8 DWT sub-bands for comparison."""
    gray = (0.299 * frames_rgb[..., 0].astype(np.float32) +
            0.587 * frames_rgb[..., 1].astype(np.float32) +
            0.114 * frames_rgb[..., 2].astype(np.float32)) / 255.0
    
    energies = {}
    for t_filt, t_name in [(SYM2_LO, 'L'), (SYM2_HI, 'H')]:
        t_out = _conv1d_strided(gray, t_filt, axis=0)
        for h_filt, h_name in [(SYM2_LO, 'L'), (SYM2_HI, 'H')]:
            h_out = _conv1d_strided(t_out, h_filt, axis=1)
            for w_filt, w_name in [(SYM2_LO, 'L'), (SYM2_HI, 'H')]:
                w_out = _conv1d_strided(h_out, w_filt, axis=2)
                name = t_name + h_name + w_name
                energies[name] = float(np.mean(w_out ** 2))
    return energies


def frame_diff_energy(frames_rgb: np.ndarray) -> Tuple[float, np.ndarray]:
    """Simple absolute frame difference — baseline temporal change measure."""
    gray = np.mean(frames_rgb.astype(np.float32), axis=-1) / 255.0
    diffs = np.abs(np.diff(gray, axis=0))          # (T-1, H, W)
    return float(np.mean(diffs)), diffs


# ─── Self-test ───────────────────────────────────────────────────────────────
_t = np.random.randint(0, 255, (32, 64, 64, 3), dtype=np.uint8)
_e, _hll = compute_hll(_t)
_de, _dd  = frame_diff_energy(_t)
print(f"✅ compute_hll     → energy={_e:.6f}, hll shape={_hll.shape}")
print(f"✅ frame_diff      → energy={_de:.6f}, diff shape={_dd.shape}")
print(f"✅ all_subbands    → {list(compute_all_subbands(_t).keys())}")


In [ ]:
# ─── Video utility: extract N evenly-spaced frames from a video ──────────────
def extract_frames(video_path: str,
                   n_frames: int = 32,
                   face_size: int = 224,
                   use_face_crop: bool = False) -> Optional[np.ndarray]:
    """
    Extract n_frames evenly spaced from video.
    
    Args:
        video_path:    path to .mp4 file
        n_frames:      how many frames to extract (default 32 — model standard)
        face_size:     resize to this square size
        use_face_crop: if True, attempt center-crop to face region
    Returns:
        frames: (n_frames, face_size, face_size, 3) uint8 or None on failure
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total < n_frames:
        cap.release()
        return None
    
    indices = np.linspace(0, total - 1, n_frames, dtype=int)
    frames = []
    
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        if use_face_crop:
            h, w = frame.shape[:2]
            margin = 0.1
            y1 = int(h * margin)
            y2 = int(h * (1 - margin))
            x1 = int(w * 0.15)
            x2 = int(w * 0.85)
            frame = frame[y1:y2, x1:x2]
        
        frame = cv2.resize(frame, (face_size, face_size))
        frames.append(frame)
    
    cap.release()
    
    if len(frames) < n_frames // 2:   # too many failed reads
        return None
    
    # Pad with last frame if slightly short
    while len(frames) < n_frames:
        frames.append(frames[-1])
    
    return np.stack(frames[:n_frames], axis=0)   # (T, H, W, 3)


# ─── HLL trajectory: per-frame-pair HLL energy ───────────────────────────────
def hll_trajectory(frames_rgb: np.ndarray, window: int = 2) -> np.ndarray:
    """
    Compute HLL energy over sliding temporal windows.
    Returns an energy value per time step, revealing dynamics.
    """
    T = frames_rgb.shape[0]
    energies = []
    for start in range(0, T - window, 1):
        chunk = frames_rgb[start : start + window + 1]  # (window+1, H, W, 3)
        e, _ = compute_hll(chunk)
        energies.append(e)
    return np.array(energies)

print("✅ Video utilities ready")


## Section 3 — Dataset Discovery

Discovering available videos on Kaggle.  
**FF++ expected:** `/kaggle/input/faceforensics/` (6000 fake + 1000 real)  
**Celeb-DF expected:** `/kaggle/input/celeb-df-v2/`

In [ ]:
# ─── Auto-discover dataset paths (handles Kaggle /datasets/ nesting) ─────────
KAGGLE_INPUT = Path('/kaggle/input')

def find_mp4s_quick(root: Path, max_depth: int = 6) -> list:
    """Recursively find .mp4 files up to max_depth."""
    results = []
    try:
        for p in root.rglob('*.mp4'):
            results.append(p)
            if len(results) >= 5:   # early-exit for scanning
                break
    except Exception:
        pass
    return results

print("📂 Scanning /kaggle/input for all directories ...", flush=True)
for d in sorted(KAGGLE_INPUT.rglob('*')):
    if d.is_dir() and d.stat().st_size == 0:  # skip empty
        pass
    # Only print directories that directly contain files
    try:
        files = list(d.glob('*.mp4'))[:1]
        if files:
            print(f"  {str(d)}")
    except Exception:
        pass

# ─── FF++ : handle both flat and nested structures ────────────────────────────
FF_METHODS = ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures',
              'FaceShifter', 'DeepFakeDetection']

# Known path fragments — we search for any dir containing these method folders
def locate_ff_root(base: Path) -> Path | None:
    """Walk the tree and find the directory that contains FF++ method folders."""
    # Check explicit known path first (from user-provided paths)
    known = base / 'datasets' / 'xdxd003' / 'ff-c23' / 'FaceForensics++_C23'
    if known.exists():
        return known
    # Generic: any directory that contains at least 2 known FF++ method subfolders
    for d in sorted(base.rglob('*')):
        if d.is_dir():
            hits = sum(1 for m in FF_METHODS if (d / m).exists())
            if hits >= 2:
                return d
    return None

FF_ROOT = locate_ff_root(KAGGLE_INPUT)
print(f"\n{'='*60}")
print(f"FF++ root: {FF_ROOT}")

FF_VIDEOS = {}
if FF_ROOT is not None:
    # Real videos
    real_paths = (list(FF_ROOT.rglob('original*/*.mp4')) +
                  list(FF_ROOT.rglob('original/*.mp4')))
    if not real_paths:
        real_paths = [p for p in FF_ROOT.rglob('*.mp4')
                      if 'original' in str(p).lower()]
    FF_VIDEOS['real'] = sorted(real_paths)

    # Fake videos by method
    for method in FF_METHODS:
        paths = list((FF_ROOT / method).glob('*.mp4')) if (FF_ROOT / method).exists() else []
        if not paths:
            paths = list(FF_ROOT.rglob(f'{method}/*.mp4'))
        if paths:
            FF_VIDEOS[method] = sorted(paths)

    for k, v in FF_VIDEOS.items():
        print(f"  FF++/{k:25s}: {len(v):5d} videos")
else:
    print("⚠️  FF++ video dataset not found.")

# ─── Also check for the extracted-frames dataset ─────────────────────────────
FRAMES_ROOT = None
known_frames = KAGGLE_INPUT / 'datasets' / 'adham7elmy' / 'faceforencispp-extracted-frames'
if known_frames.exists():
    FRAMES_ROOT = known_frames
    print(f"\n✅ Found extracted-frames dataset: {FRAMES_ROOT}")
    fake_methods_frames = [d.name for d in (FRAMES_ROOT / 'fake').iterdir()
                           if d.is_dir()] if (FRAMES_ROOT / 'fake').exists() else []
    print(f"   Fake methods (frames): {fake_methods_frames}")
else:
    for d in KAGGLE_INPUT.rglob('*'):
        if d.is_dir() and (d / 'fake').exists() and (d / 'real').exists():
            FRAMES_ROOT = d
            print(f"\n✅ Found extracted-frames dataset: {FRAMES_ROOT}")
            break


In [ ]:
# ─── Celeb-DF paths (handles /datasets/ nesting) ─────────────────────────────
def locate_celeb_root(base: Path) -> Path | None:
    """Find Celeb-DF root — looks for Celeb-real and Celeb-synthesis subfolders."""
    known = base / 'datasets' / 'reubensuju' / 'celeb-df-v2'
    if known.exists():
        return known
    for d in sorted(base.rglob('*')):
        if d.is_dir():
            if (d / 'Celeb-real').exists() and (d / 'Celeb-synthesis').exists():
                return d
    return None

CELEB_ROOT = locate_celeb_root(KAGGLE_INPUT)
print(f"Celeb-DF root: {CELEB_ROOT}")

CELEB_VIDEOS = {}
if CELEB_ROOT is not None:
    real_paths  = (list((CELEB_ROOT / 'Celeb-real').glob('*.mp4')) +
                   list((CELEB_ROOT / 'YouTube-real').glob('*.mp4')))
    fake_paths  = list((CELEB_ROOT / 'Celeb-synthesis').glob('*.mp4'))
    CELEB_VIDEOS['real'] = sorted(real_paths)
    CELEB_VIDEOS['fake'] = sorted(fake_paths)
    for k, v in CELEB_VIDEOS.items():
        print(f"  Celeb-DF/{k:10s}: {len(v):5d} videos")
    
    # Check test split file
    test_list = CELEB_ROOT / 'List_of_testing_videos.txt'
    if test_list.exists():
        with open(test_list) as f:
            test_names = set(line.strip().split()[-1].split('/')[-1].replace('.mp4','')
                           for line in f if line.strip())
        print(f"  Test split file found: {len(test_names)} test videos")
        # Keep test videos separate
        CELEB_TEST_REAL = [p for p in CELEB_VIDEOS['real'] if p.stem in test_names]
        CELEB_TEST_FAKE = [p for p in CELEB_VIDEOS['fake'] if p.stem in test_names]
        CELEB_TRAIN_REAL = [p for p in CELEB_VIDEOS['real'] if p.stem not in test_names]
        CELEB_TRAIN_FAKE = [p for p in CELEB_VIDEOS['fake'] if p.stem not in test_names]
        print(f"  Test:  {len(CELEB_TEST_REAL)} real, {len(CELEB_TEST_FAKE)} fake")
        print(f"  Train: {len(CELEB_TRAIN_REAL)} real, {len(CELEB_TRAIN_FAKE)} fake")
else:
    print("⚠️  Celeb-DF not found. Add: reubensuju/celeb-df-v2")
    CELEB_VIDEOS = {}


In [ ]:
# ─── Sampling helpers + frames-dataset fallback ───────────────────────────────
N_SAMPLE = 20   # Videos per class/method

def sample_videos(video_list: list, n: int = N_SAMPLE, seed: int = SEED) -> list:
    rng = random.Random(seed)
    return rng.sample(video_list, min(n, len(video_list)))

# ─── Frames dataset: load PNG sequences instead of videos ─────────────────────
def load_frames_from_dir(frame_dir: Path, n_frames: int = 32,
                         face_size: int = 112) -> Optional[np.ndarray]:
    pngs = sorted(frame_dir.glob('*.png'))
    if len(pngs) < 4:
        return None
    indices = np.linspace(0, len(pngs) - 1, n_frames, dtype=int)
    frames = []
    for idx in indices:
        img = cv2.imread(str(pngs[idx]))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (face_size, face_size))
        frames.append(img)
    if len(frames) < n_frames // 2:
        return None
    while len(frames) < n_frames:
        frames.append(frames[-1])
    return np.stack(frames[:n_frames], axis=0)

def get_frame_dirs(frames_root: Path, split: str, method: str = None) -> list:
    if frames_root is None:
        return []
    base = frames_root / split
    if not base.exists():
        return []
    if method:
        method_dir = base / method
        return sorted([d for d in method_dir.iterdir() if d.is_dir()]) if method_dir.exists() else []
    else:
        return sorted([d for d in base.iterdir() if d.is_dir()])

def load_clip(source, n_frames=32, face_size=112) -> Optional[np.ndarray]:
    """Load a clip from either a .mp4 path or a frames directory."""
    if isinstance(source, Path) and source.is_dir():
        return load_frames_from_dir(source, n_frames, face_size)
    return extract_frames(str(source), n_frames, face_size)

# ─── Build unified SOURCES dict (video paths OR frame dirs) ───────────────────
SOURCES = {}

# Priority: use video files if found by discovery
for k, v in FF_VIDEOS.items():
    SOURCES[k] = v

# Fallback: use extracted-frames dataset if no videos found
if not SOURCES and FRAMES_ROOT is not None:
    print("INFO: No video files found — using extracted-frames dataset instead")
    SOURCES['real'] = get_frame_dirs(FRAMES_ROOT, 'real')
    for method in ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']:
        dirs = get_frame_dirs(FRAMES_ROOT, 'fake', method)
        if dirs:
            SOURCES[method] = dirs

print("Available sources:")
for k, v in SOURCES.items():
    print(f"  {k:<25}: {len(v):5d} items")

# Quick sanity test
if SOURCES:
    first_key = next(iter(SOURCES))
    test_src = SOURCES[first_key][0]
    test_frames = load_clip(test_src, n_frames=32, face_size=112)
    if test_frames is not None:
        e, hll = compute_hll(test_frames)
        print(f"\nLoad test ({first_key}): frames={test_frames.shape}, HLL energy={e:.6f}")
    else:
        print("\nLoad test failed — check dataset structure")


## Section 4 — Baseline: Frame Difference (Sanity Check)

**Before DWT:** does raw frame-to-frame difference separate real from fake?

This is the absolute baseline. If frame difference doesn't separate the classes,
we have a data problem, not a model problem. If it does, then temporal signals
are real and the question becomes whether DWT adds anything.

In [ ]:
# ─── Collect frame difference energies ───────────────────────────────────────
def analyze_videos(video_paths: list, label: str, n_sample: int = N_SAMPLE,
                   verbose: bool = True) -> Dict:
    """
    Process a list of videos and compute HLL + frame-diff metrics.
    Returns dict with per-video results.
    """
    sampled = sample_videos(video_paths, n_sample)
    results = []
    
    for i, vpath in enumerate(sampled):
        frames = load_clip(vpath, n_frames=32, face_size=112)
        if frames is None:
            if verbose: print(f"  ⚠️  Skip {Path(vpath).name} (read failed)")
            continue
        
        hll_e, hll_map      = compute_hll(frames)
        fd_e, _             = frame_diff_energy(frames)
        all_bands           = compute_all_subbands(frames)
        traj                = hll_trajectory(frames, window=2)
        
        results.append({
            'video':       Path(vpath).name,
            'label':       label,
            'hll_energy':  hll_e,
            'fd_energy':   fd_e,
            'hll_traj':    traj,
            'hll_map':     hll_map,
            'all_bands':   all_bands,
        })
        if verbose and (i + 1) % 5 == 0:
            print(f"  [{label}] {i+1}/{len(sampled)} — hll={hll_e:.6f}, fd={fd_e:.6f}")
    
    return results

print("📊 Collecting metrics... (this takes a few minutes)")
print("─" * 60)

all_results = {}

# Real videos (FF++)
if 'real' in SOURCES and SOURCES['real']:
    all_results['real'] = analyze_videos(SOURCES['real'], 'real')
    print(f"  ✅ real: {len(all_results['real'])} videos processed\n")

# Each manipulation method
for method in ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']:
    if method in SOURCES and SOURCES[method]:
        all_results[method] = analyze_videos(SOURCES[method], method)
        print(f"  ✅ {method}: {len(all_results[method])} videos processed\n")

print("─" * 60)
print(f"✅ Total groups collected: {list(all_results.keys())}")


In [ ]:
# ─── Plot: Frame Difference distributions ─────────────────────────────────────
COLORS = {
    'real':           '#2ecc71',
    'Deepfakes':      '#e74c3c',
    'Face2Face':      '#e67e22',
    'FaceSwap':       '#9b59b6',
    'NeuralTextures': '#3498db',
    'FaceShifter':    '#1abc9c',
}

def get_values(results_dict: dict, key: str) -> Dict[str, list]:
    return {k: [r[key] for r in v] for k, v in results_dict.items()}

fd_vals = get_values(all_results, 'fd_energy')

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Section 4: Frame Difference Baseline\n(Does temporal change separate real from fake?)',
             fontsize=14, fontweight='bold')

# ── Boxplot ──────────────────────────────────────────────────────────────────
ax = axes[0]
groups  = list(fd_vals.keys())
data    = [fd_vals[g] for g in groups]
colors  = [COLORS.get(g, '#95a5a6') for g in groups]

bp = ax.boxplot(data, patch_artist=True, notch=True,
                medianprops=dict(color='black', linewidth=2))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_xticklabels([g.replace('NeuralTextures', 'NeuralTex.') for g in groups], rotation=30, ha='right')
ax.set_ylabel('Mean Absolute Frame Difference')
ax.set_title('Frame Difference Energy by Class')
ax.grid(True, alpha=0.3)

# Annotate medians
for i, (g, vals) in enumerate(zip(groups, data)):
    med = np.median(vals)
    ax.text(i + 1, med * 1.05, f'{med:.4f}', ha='center', fontsize=8, color='black')

# ── Violin ───────────────────────────────────────────────────────────────────
ax = axes[1]
positions = np.arange(1, len(groups) + 1)
parts = ax.violinplot(data, positions=positions, showmedians=True, showextrema=True)
for i, (pc, color) in enumerate(zip(parts['bodies'], colors)):
    pc.set_facecolor(color)
    pc.set_alpha(0.6)

ax.set_xticks(positions)
ax.set_xticklabels([g.replace('NeuralTextures', 'NeuralTex.') for g in groups], rotation=30, ha='right')
ax.set_ylabel('Mean Absolute Frame Difference')
ax.set_title('Distribution Shape (Violin)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'sec4_frame_diff_baseline.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved sec4_frame_diff_baseline.png")

# ── Mann-Whitney U test: each fake method vs real ────────────────────────────
print("\n📊 Statistical Significance (Mann-Whitney U, real vs each method):")
print(f"{'Method':<20} {'Real median':>12} {'Fake median':>12} {'p-value':>12} {'Significant?':>13}")
print("─" * 70)
real_fd = fd_vals.get('real', [])
for method, vals in fd_vals.items():
    if method == 'real': continue
    stat, p = stats.mannwhitneyu(real_fd, vals, alternative='two-sided')
    sig = '✅ YES' if p < 0.05 else '❌ NO'
    print(f"{method:<20} {np.median(real_fd):>12.5f} {np.median(vals):>12.5f} {p:>12.4f} {sig:>13}")


## Section 5 — HLL Energy Analysis (Primary Hypothesis Test)

Now we apply the 3D-DWT and measure the HLL sub-band energy directly.

**Key questions:**
1. Is HLL energy higher for fakes than real? (magnitude test)
2. Which manipulation method shows the strongest signal?
3. Is HLL better than raw frame difference?
4. Are all 8 DWT sub-bands discriminative, or just HLL?

In [ ]:
# ─── HLL Energy boxplots ─────────────────────────────────────────────────────
hll_vals = get_values(all_results, 'hll_energy')

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Section 5: HLL Sub-Band Energy\n(Primary Hypothesis: Deepfakes have elevated temporal flicker)',
             fontsize=14, fontweight='bold')

groups = list(hll_vals.keys())
data   = [hll_vals[g] for g in groups]
colors = [COLORS.get(g, '#95a5a6') for g in groups]

# ── Boxplot ──────────────────────────────────────────────────────────────────
ax = axes[0]
bp = ax.boxplot(data, patch_artist=True, notch=False,
                medianprops=dict(color='black', linewidth=2.5))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.75)

ax.set_xticklabels([g.replace('NeuralTextures', 'NeuralTex.') for g in groups], rotation=30, ha='right')
ax.set_ylabel('HLL Energy (mean squared coefficient)')
ax.set_title('HLL Energy per Class')
ax.grid(True, alpha=0.3)

for i, (g, vals) in enumerate(zip(groups, data)):
    med = np.median(vals)
    ax.text(i + 1, med * 1.02, f'{med:.5f}', ha='center', fontsize=8)

# ── Effect size: HLL vs Frame Diff comparison ─────────────────────────────────
ax = axes[1]
real_hll = hll_vals.get('real', [])
real_fd  = fd_vals.get('real', [])

methods_plot = [g for g in groups if g != 'real']
x = np.arange(len(methods_plot))
w = 0.35

hll_ratios = []
fd_ratios  = []
for method in methods_plot:
    fake_hll = hll_vals.get(method, [1])
    fake_fd  = fd_vals.get(method, [1])
    hll_ratios.append(np.median(fake_hll) / max(np.median(real_hll), 1e-10))
    fd_ratios.append(np.median(fake_fd)   / max(np.median(real_fd),  1e-10))

bars1 = ax.bar(x - w/2, hll_ratios, w, label='HLL ratio (fake/real)', color='#e74c3c', alpha=0.75)
bars2 = ax.bar(x + w/2, fd_ratios,  w, label='FrameDiff ratio (fake/real)', color='#3498db', alpha=0.75)
ax.axhline(1.0, color='black', linestyle='--', label='ratio=1 (no difference)')
ax.set_xticks(x)
ax.set_xticklabels([m.replace('NeuralTextures', 'NeuralTex.') for m in methods_plot], rotation=30, ha='right')
ax.set_ylabel('Median energy ratio (fake / real)')
ax.set_title('HLL vs Frame Diff: Which is more discriminative?')
ax.legend()
ax.grid(True, alpha=0.3)

# Annotate
for bar, val in zip(bars1, hll_ratios):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, f'{val:.2f}x',
            ha='center', va='bottom', fontsize=8, color='#e74c3c', fontweight='bold')

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'sec5_hll_energy.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved sec5_hll_energy.png")

# ── Stats table ───────────────────────────────────────────────────────────────
print("\n📊 HLL Energy Statistics:")
print(f"{'Class':<20} {'Mean':>10} {'Median':>10} {'Std':>10} {'vs Real':>10} {'p-value':>10}")
print("─" * 72)
for method, vals in hll_vals.items():
    ratio = np.median(vals) / max(np.median(real_hll), 1e-10) if method != 'real' else 1.0
    if method != 'real' and real_hll:
        _, p = stats.mannwhitneyu(real_hll, vals, alternative='two-sided')
        p_str = f'{p:.4f}'
    else:
        p_str = '—'
    print(f"{method:<20} {np.mean(vals):>10.6f} {np.median(vals):>10.6f} {np.std(vals):>10.6f} {ratio:>10.2f}x {p_str:>10}")


In [ ]:
# ─── All 8 sub-bands: which are discriminative? ───────────────────────────────
subband_names = ['LLL', 'LLH', 'LHL', 'LHH', 'HLL', 'HLH', 'HHL', 'HHH']

# Aggregate per sub-band per class
subband_data = {}
for method, results in all_results.items():
    sb = {name: [] for name in subband_names}
    for r in results:
        for name in subband_names:
            sb[name].append(r['all_bands'].get(name, 0.0))
    subband_data[method] = {name: np.median(vals) for name, vals in sb.items()}

# Compute ratio: each fake method median / real median per subband
real_sb = subband_data.get('real', {})
fig, ax = plt.subplots(figsize=(14, 6))
fig.suptitle('Section 5b: All 8 DWT Sub-Bands — Which Discriminate Real vs Fake?',
             fontsize=13, fontweight='bold')

x = np.arange(len(subband_names))
width = 0.8 / max(1, len(all_results) - 1)

fake_methods = [m for m in all_results if m != 'real']
offsets = np.linspace(-0.35, 0.35, len(fake_methods))

for i, method in enumerate(fake_methods):
    ratios = [subband_data[method].get(sb, 0) / max(real_sb.get(sb, 1e-10), 1e-10)
              for sb in subband_names]
    bars = ax.bar(x + offsets[i], ratios, width * 0.9,
                  label=method, color=COLORS.get(method, '#95a5a6'), alpha=0.75)

ax.axhline(1.0, color='black', linewidth=1.5, linestyle='--', label='no difference')
ax.set_xticks(x)
ax.set_xticklabels(subband_names, fontsize=12, fontweight='bold')

# Highlight HLL
ax.axvspan(4 - 0.45, 4 + 0.45, alpha=0.1, color='red', label='HLL (our target)')
ax.text(4, ax.get_ylim()[1] * 0.95 if ax.get_ylim()[1] > 1 else 1.5,
        'HLL ↓', ha='center', fontsize=11, color='red', fontweight='bold')

ax.set_ylabel('Median Energy Ratio (fake / real)')
ax.set_xlabel('DWT Sub-band (Temporal × Height × Width frequency)')
ax.legend(loc='upper right', fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'sec5b_all_subbands.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved sec5b_all_subbands.png")
print("\n💡 Key insight: if HLL bar is tallest → HLL is uniquely discriminative")
print("   If LLL or spatial bands are equally tall → spatial features matter more than temporal")


## Section 6 — Temporal Trajectory Analysis

A scalar energy value per video is useful, but **how does HLL energy change over time** 
within a video reveals the temporal pattern.

**Real videos:** HLL should fluctuate naturally with motion/lighting — irregular, random-looking.  
**Deepfakes:** HLL should show more structured patterns — sustained elevation, periodic spikes.

In [ ]:
# ─── Plot temporal trajectories ──────────────────────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Section 6: Temporal HLL Trajectory\n(How does flicker evolve across frames?)',
             fontsize=14, fontweight='bold')
axes = axes.flatten()

plot_methods = ['real'] + [m for m in ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures'] 
                            if m in all_results][:4]

for ax_idx, method in enumerate(plot_methods):
    ax = axes[ax_idx]
    results = all_results.get(method, [])[:8]   # Show max 8 videos
    color = COLORS.get(method, '#95a5a6')
    
    all_trajs = []
    for r in results:
        traj = r['hll_traj']
        if len(traj) > 0:
            # Normalize each trajectory to [0,1] for shape comparison
            traj_norm = (traj - traj.min()) / max(traj.max() - traj.min(), 1e-10)
            all_trajs.append(traj_norm)
            ax.plot(traj_norm, alpha=0.4, linewidth=1, color=color)
    
    if all_trajs:
        min_len = min(len(t) for t in all_trajs)
        stacked = np.stack([t[:min_len] for t in all_trajs], axis=0)
        mean_traj = stacked.mean(axis=0)
        std_traj  = stacked.std(axis=0)
        x = np.arange(min_len)
        ax.plot(mean_traj, color='black', linewidth=2.5, label='Mean trajectory')
        ax.fill_between(x, mean_traj - std_traj, mean_traj + std_traj,
                        alpha=0.2, color='black', label='±1 std')
    
    # Compute trajectory statistics
    if all_trajs:
        raw_trajs = [r['hll_traj'] for r in results if len(r['hll_traj']) > 0]
        variances  = [np.var(t) for t in raw_trajs]
        mean_var   = np.mean(variances)
    else:
        mean_var = 0
    
    ax.set_title(f'{method}\n(mean variance = {mean_var:.2e})', fontweight='bold', color=color)
    ax.set_xlabel('Time window index')
    ax.set_ylabel('Normalized HLL energy')
    ax.set_ylim(-0.05, 1.1)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

# Hide unused axes
for ax_idx in range(len(plot_methods), len(axes)):
    axes[ax_idx].set_visible(False)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'sec6_temporal_trajectory.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved sec6_temporal_trajectory.png")


In [ ]:
# ─── Trajectory variance comparison ──────────────────────────────────────────
# Higher variance = more erratic HLL = potentially more suspicious?
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Section 6b: HLL Trajectory Statistics', fontsize=13, fontweight='bold')

groups = list(all_results.keys())

# Compute autocorrelation at lag-1 (how predictable is the next flicker value?)
def lag1_autocorr(x):
    if len(x) < 3: return 0.0
    return np.corrcoef(x[:-1], x[1:])[0, 1]

variances = {g: [np.var(r['hll_traj']) for r in all_results[g] if len(r['hll_traj']) > 0]
             for g in groups}
autocorrs = {g: [lag1_autocorr(r['hll_traj']) for r in all_results[g] if len(r['hll_traj']) > 0]
             for g in groups}

# Variance boxplot
ax = axes[0]
data   = [variances.get(g, [0]) for g in groups]
colors = [COLORS.get(g, '#95a5a6') for g in groups]
bp = ax.boxplot(data, patch_artist=True, medianprops=dict(color='black', linewidth=2))
for patch, c in zip(bp['boxes'], colors): patch.set_facecolor(c); patch.set_alpha(0.7)
ax.set_xticklabels([g.replace('NeuralTextures','NeuralTex.') for g in groups], rotation=30, ha='right')
ax.set_ylabel('HLL trajectory variance')
ax.set_title('Variance: Higher = More Erratic Flicker')
ax.grid(True, alpha=0.3)

# Autocorrelation boxplot
ax = axes[1]
data = [autocorrs.get(g, [0]) for g in groups]
bp = ax.boxplot(data, patch_artist=True, medianprops=dict(color='black', linewidth=2))
for patch, c in zip(bp['boxes'], colors): patch.set_facecolor(c); patch.set_alpha(0.7)
ax.set_xticklabels([g.replace('NeuralTextures','NeuralTex.') for g in groups], rotation=30, ha='right')
ax.set_ylabel('Lag-1 autocorrelation of HLL trajectory')
ax.set_title('Autocorrelation: Higher = More Structured/Predictable Flicker')
ax.axhline(0, color='black', linestyle='--', linewidth=1)
ax.grid(True, alpha=0.3)

print("\n📊 Trajectory variance (mean per class):")
for g in groups:
    vals = variances.get(g, [])
    if vals:
        print(f"  {g:<20}: {np.mean(vals):.2e}")

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'sec6b_trajectory_stats.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved sec6b_trajectory_stats.png")


## Section 7 — Spatial HLL Heatmaps

*Where* in the face does temporal flicker concentrate?

Real: flicker should be diffuse (camera noise, natural motion).  
Deepfakes: flicker should concentrate at **blend boundaries** — typically the face oval boundary.

In [ ]:
# ─── Average HLL spatial heatmap per class ────────────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Section 7: Spatial HLL Heatmaps\n(Where does temporal flicker concentrate?)',
             fontsize=14, fontweight='bold')
axes = axes.flatten()

plot_methods = ['real'] + [m for m in ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']
                            if m in all_results][:4]

global_vmax = 0
avg_maps = {}

for method in plot_methods:
    results = all_results.get(method, [])
    maps = []
    for r in results:
        hll_map = np.abs(r['hll_map'])              # (T//2, H//2, W//2)
        temporal_max = hll_map.max(axis=0)           # (H//2, W//2) — worst-case per pixel
        maps.append(temporal_max)
    if maps:
        avg_map = np.mean(maps, axis=0)              # average over videos
        avg_maps[method] = avg_map
        global_vmax = max(global_vmax, avg_map.max())

for ax_idx, method in enumerate(plot_methods):
    ax = axes[ax_idx]
    if method not in avg_maps:
        ax.set_visible(False)
        continue
    
    avg_map = avg_maps[method]
    color = COLORS.get(method, '#95a5a6')
    
    im = ax.imshow(avg_map, cmap='hot', vmin=0, vmax=global_vmax, interpolation='bilinear')
    plt.colorbar(im, ax=ax, fraction=0.046)
    ax.set_title(f'{method}\nMax HLL={avg_map.max():.5f}', fontweight='bold',
                 color='white' if method != 'real' else 'black',
                 bbox=dict(facecolor=color, alpha=0.8, boxstyle='round'))
    ax.axis('off')

for ax_idx in range(len(plot_methods), len(axes)):
    axes[ax_idx].set_visible(False)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'sec7_spatial_heatmaps.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved sec7_spatial_heatmaps.png")
print("\n💡 Key: if deepfakes show ring-shaped or boundary-concentrated heatmaps → HLL detects blend edges")
print("         if all heatmaps look similar → HLL doesn't localize forgeries spatially")


In [ ]:
# ─── Center vs border HLL ratio ──────────────────────────────────────────────
# Blend boundaries are at the face periphery, so fake videos should have
# higher HLL at borders vs center compared to real
print("📊 Center vs Border HLL concentration:")
print(f"{'Class':<20} {'Center':>10} {'Border':>10} {'Border/Center':>15}")
print("─" * 55)

for method in plot_methods:
    if method not in avg_maps:
        continue
    avg_map = avg_maps[method]
    H, W    = avg_map.shape
    
    # Center region: middle 50%
    cy1, cy2 = H // 4, 3 * H // 4
    cx1, cx2 = W // 4, 3 * W // 4
    center_val = avg_map[cy1:cy2, cx1:cx2].mean()
    
    # Border: everything outside center
    border_mask = np.ones_like(avg_map, dtype=bool)
    border_mask[cy1:cy2, cx1:cx2] = False
    border_val  = avg_map[border_mask].mean()
    
    ratio = border_val / max(center_val, 1e-10)
    print(f"{method:<20} {center_val:>10.5f} {border_val:>10.5f} {ratio:>15.3f}x")


## Section 8 — Cross-Dataset Validation (Celeb-DF)

**The generalization test:** If HLL energy separates real from fake on Celeb-DF  
(which uses *different generation methods* than FF++), then HLL is a universal signal.

If the HLL distributions on Celeb-DF look similar to FF++, we're on the right track for V8.0.

In [ ]:
# ─── Celeb-DF HLL analysis ───────────────────────────────────────────────────
celeb_results = {}

if CELEB_VIDEOS:
    print("📊 Processing Celeb-DF videos...")
    for split in ['real', 'fake']:
        if split in CELEB_VIDEOS and CELEB_VIDEOS[split]:
            celeb_results[f'CelebDF_{split}'] = analyze_videos(
                CELEB_VIDEOS[split], f'CelebDF_{split}', n_sample=N_SAMPLE
            )
            print(f"  ✅ CelebDF/{split}: {len(celeb_results[f'CelebDF_{split}'])} videos")
else:
    print("⚠️  Celeb-DF not available — skipping cross-dataset section")
    print("   To enable: attach 'andrewmvd/celeb-df-v2' to your Kaggle notebook")

if celeb_results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle('Section 8: Cross-Dataset Generalization\nDoes HLL signal transfer from FF++ to Celeb-DF?',
                 fontsize=13, fontweight='bold')
    
    CELEB_COLORS = {'CelebDF_real': '#2ecc71', 'CelebDF_fake': '#e74c3c'}
    all_for_plot  = {**hll_vals}
    celeb_hll     = {k: [r['hll_energy'] for r in v] for k, v in celeb_results.items()}
    all_for_plot.update(celeb_hll)
    
    # Boxplot: FF++ methods + Celeb-DF
    ax = axes[0]
    groups = list(all_for_plot.keys())
    data   = [all_for_plot[g] for g in groups]
    colors = [CELEB_COLORS.get(g, COLORS.get(g, '#95a5a6')) for g in groups]
    
    bp = ax.boxplot(data, patch_artist=True, medianprops=dict(color='black', linewidth=2))
    for patch, c in zip(bp['boxes'], colors): patch.set_facecolor(c); patch.set_alpha(0.7)
    ax.set_xticklabels([g.replace('NeuralTextures','NeuralTex.').replace('CelebDF_','CDF-')
                        for g in groups], rotation=40, ha='right', fontsize=9)
    ax.set_ylabel('HLL Energy')
    ax.set_title('HLL Energy: FF++ vs Celeb-DF')
    ax.axvline(len(hll_vals) + 0.5, color='gray', linestyle='--', alpha=0.7)
    ax.text(len(hll_vals) + 0.6, ax.get_ylim()[1] * 0.95, 'Celeb-DF →', fontsize=9, color='gray')
    ax.grid(True, alpha=0.3)
    
    # Compare: does the real/fake gap transfer?
    ax = axes[1]
    ff_real_med   = np.median(hll_vals.get('real', [0]))
    ff_fake_meds  = {m: np.median(hll_vals.get(m, [0])) for m in hll_vals if m != 'real'}
    cdf_real_med  = np.median(celeb_hll.get('CelebDF_real', [0]))
    cdf_fake_med  = np.median(celeb_hll.get('CelebDF_fake', [0]))
    
    datasets = ['FF++ Real'] + [f'FF++\n{m[:8]}' for m in ff_fake_meds] + ['CDF Real', 'CDF Fake']
    values   = [ff_real_med] + list(ff_fake_meds.values()) + [cdf_real_med, cdf_fake_med]
    bar_colors = ['#2ecc71'] + [COLORS.get(m, '#e74c3c') for m in ff_fake_meds] + ['#2ecc71', '#e74c3c']
    
    bars = ax.bar(range(len(values)), values, color=bar_colors, alpha=0.75, edgecolor='white')
    ax.set_xticks(range(len(values)))
    ax.set_xticklabels(datasets, rotation=40, ha='right', fontsize=8)
    ax.set_ylabel('Median HLL Energy')
    ax.set_title('Median HLL: Does the Gap Transfer to Celeb-DF?')
    ax.grid(True, alpha=0.3, axis='y')
    
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, val * 1.02, f'{val:.5f}',
                ha='center', va='bottom', fontsize=7, rotation=45)
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'sec8_cross_dataset.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ Saved sec8_cross_dataset.png")


## Section 9 — SBI vs Real Deepfake Comparison

**Critical question for V8.0 training strategy:** Is SBI a good proxy for real deepfakes?

We generate simple SBI examples in-notebook (no dlib required — basic blend only)
and compare their HLL energy to real FF++ deepfakes.

If SBI HLL ≈ real deepfake HLL → SBI training is valid.  
If SBI HLL ≫ real deepfake HLL → SBI creates unrealistic artifacts → training fails.

In [ ]:
# ─── Simple SBI generator (no landmarks — basic alpha blend) ─────────────────
def generate_simple_sbi(frame: np.ndarray, blend_alpha: float = 0.4) -> np.ndarray:
    """
    Simplest possible SBI: blend face with a slightly shifted copy.
    No landmarks needed — just center region blending.
    This is a crude proxy but enough for HLL comparison.
    """
    H, W = frame.shape[:2]
    fake = frame.copy()
    
    # Center crop = 'face region'
    margin = 0.15
    y1, y2 = int(H * margin), int(H * (1 - margin))
    x1, x2 = int(W * margin), int(W * (1 - margin))
    
    # Perturb: slight elastic warp on source
    dx = np.random.randint(3, 8)
    dy = np.random.randint(3, 8)
    source_region = frame[y1+dy:y2+dy, x1+dx:x2+dx]
    target_region = frame[y1:y2, x1:x2]
    
    # Resize source to target if needed
    if source_region.shape != target_region.shape:
        source_region = cv2.resize(source_region, (target_region.shape[1], target_region.shape[0]))
    
    # Blend with smooth mask
    blended = (blend_alpha * source_region + (1 - blend_alpha) * target_region).astype(np.uint8)
    
    # Feather the edges
    mask = np.zeros((y2 - y1, x2 - x1), dtype=np.float32)
    inner_margin = 10
    mask[inner_margin:-inner_margin, inner_margin:-inner_margin] = 1.0
    mask = cv2.GaussianBlur(mask, (21, 21), 0)
    mask = mask[:, :, np.newaxis]
    
    fake[y1:y2, x1:x2] = (mask * blended + (1 - mask) * target_region).astype(np.uint8)
    return fake


def generate_sbi_video(real_frames: np.ndarray) -> np.ndarray:
    """Generate SBI fake version of a real video clip."""
    sbi_frames = []
    for frame in real_frames:
        # Small Brownian drift: ±2px per frame
        fake = generate_simple_sbi(frame, blend_alpha=np.random.uniform(0.3, 0.6))
        sbi_frames.append(fake)
    return np.stack(sbi_frames, axis=0)


# ─── Compare SBI vs real deepfake HLL ────────────────────────────────────────
sbi_results = []
real_source_results = []

if 'real' in SOURCES and SOURCES['real']:
    real_sample = sample_videos(SOURCES['real'], 10)
    print("📊 Generating SBI fakes from real videos and comparing HLL...")
    
    for vpath in real_sample:
        frames = load_clip(vpath, n_frames=32, face_size=112)
        if frames is None: continue
        
        # Real HLL
        real_e, _ = compute_hll(frames)
        real_source_results.append(real_e)
        
        # SBI HLL
        sbi_frames = generate_sbi_video(frames)
        sbi_e, _   = compute_hll(sbi_frames)
        sbi_results.append(sbi_e)
        
        print(f"  Real: {real_e:.6f} → SBI: {sbi_e:.6f} (ratio: {sbi_e/max(real_e,1e-10):.2f}x)")

print(f"\nReal HLL median:  {np.median(real_source_results):.6f}")
print(f"SBI HLL median:   {np.median(sbi_results):.6f}")
print(f"SBI/Real ratio:   {np.median(sbi_results)/max(np.median(real_source_results),1e-10):.2f}x")

ff_df_hll   = hll_vals.get('Deepfakes', [])
ff_df_med   = np.median(ff_df_hll) if ff_df_hll else None
print(f"FF++ Deepfakes HLL median: {ff_df_med:.6f}" if ff_df_med else "FF++ Deepfakes not available")


In [ ]:
# ─── SBI comparison plot ─────────────────────────────────────────────────────
if sbi_results and real_source_results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle('Section 9: SBI vs Real Deepfakes — Is SBI a Good Training Proxy?',
                 fontsize=13, fontweight='bold')
    
    ax = axes[0]
    comparison_groups  = {'Real':       real_source_results,
                          'SBI Fake':   sbi_results}
    if ff_df_hll:
        comparison_groups['FF++ Deepfakes'] = ff_df_hll
    if 'Face2Face' in hll_vals:
        comparison_groups['FF++ Face2Face'] = hll_vals['Face2Face']
    
    comp_colors = {'Real':'#2ecc71', 'SBI Fake':'#f39c12',
                   'FF++ Deepfakes':'#e74c3c', 'FF++ Face2Face':'#e67e22'}
    
    data   = list(comparison_groups.values())
    labels = list(comparison_groups.keys())
    colors = [comp_colors.get(l, '#95a5a6') for l in labels]
    
    bp = ax.boxplot(data, patch_artist=True, medianprops=dict(color='black', linewidth=2.5))
    for patch, c in zip(bp['boxes'], colors): patch.set_facecolor(c); patch.set_alpha(0.75)
    ax.set_xticklabels(labels, rotation=20, ha='right')
    ax.set_ylabel('HLL Energy')
    ax.set_title('SBI HLL vs Real Deepfake HLL')
    ax.grid(True, alpha=0.3)
    
    ax = axes[1]
    # Paired comparison: same source video, real vs SBI
    x = np.arange(len(real_source_results))
    ax.scatter(x, real_source_results, color='#2ecc71', s=80, label='Real', zorder=3)
    ax.scatter(x, sbi_results,         color='#f39c12', s=80, label='SBI Fake', zorder=3, marker='^')
    for i in range(len(real_source_results)):
        ax.plot([i, i], [real_source_results[i], sbi_results[i]],
                color='gray', alpha=0.4, linewidth=1)
    ax.axhline(np.median(real_source_results), color='#2ecc71', linestyle='--', alpha=0.7, label='Real median')
    ax.axhline(np.median(sbi_results),         color='#f39c12', linestyle='--', alpha=0.7, label='SBI median')
    if ff_df_med:
        ax.axhline(ff_df_med, color='#e74c3c', linestyle=':', alpha=0.7, label='FF++ Deepfakes median')
    ax.set_xlabel('Video index')
    ax.set_ylabel('HLL Energy')
    ax.set_title('Paired: Same Source Video — Real vs SBI')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'sec9_sbi_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ Saved sec9_sbi_comparison.png")
    
    # Key diagnostic
    sbi_med  = np.median(sbi_results)
    real_med = np.median(real_source_results)
    print("\n🔍 SBI Proxy Quality Diagnosis:")
    if ff_df_med:
        sbi_diff  = abs(sbi_med  - ff_df_med) / max(ff_df_med, 1e-10)
        real_diff = abs(real_med - ff_df_med) / max(ff_df_med, 1e-10)
        print(f"  SBI  distance from FF++ Deepfakes: {sbi_diff:.1%}")
        print(f"  Real distance from FF++ Deepfakes: {real_diff:.1%}")
        if sbi_diff < real_diff:
            print("  ✅ SBI is a reasonable proxy for real deepfakes on HLL")
        else:
            print("  ⚠️  SBI is farther from real deepfakes than real videos are!")
            print("     This explains why SBI-trained models fail on real deepfakes")


## Section 10 — Summary & V8.0 Recommendations

Based on the analysis above, fill in the findings table below.  
The answers directly determine the V8.0 architecture choices.

In [ ]:
# ─── Auto-generate findings report ───────────────────────────────────────────
print("=" * 70)
print("📋 STF-MAMBA V8.0 DATA ANALYSIS REPORT")
print("=" * 70)

real_hll_med = np.median(hll_vals.get('real', [0]))
print(f"\n1. HLL ENERGY SEPARATION")
print(f"   Real videos median HLL: {real_hll_med:.6f}")
for method in ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']:
    if method in hll_vals:
        med = np.median(hll_vals[method])
        ratio = med / max(real_hll_med, 1e-10)
        sep = '✅ SEPARATES' if ratio > 1.1 else '⚠️  WEAK' if ratio > 0.9 else '❌ INVERTED'
        print(f"   {method:<20}: {med:.6f} ({ratio:.2f}x real) → {sep}")

print(f"\n2. FRAME DIFF vs HLL")
real_fd_med = np.median(fd_vals.get('real', [0]))
for method in ['Deepfakes', 'Face2Face']:
    if method in hll_vals and method in fd_vals:
        hll_ratio = np.median(hll_vals[method]) / max(real_hll_med, 1e-10)
        fd_ratio  = np.median(fd_vals[method])  / max(real_fd_med,  1e-10)
        winner = 'HLL' if hll_ratio > fd_ratio else 'FrameDiff'
        print(f"   {method}: HLL={hll_ratio:.2f}x | FrameDiff={fd_ratio:.2f}x | Winner={winner}")

print(f"\n3. MOST DISCRIMINATIVE DWT SUB-BAND")
if all_results and 'real' in all_results:
    real_sb = {name: np.mean([r['all_bands'].get(name, 0) for r in all_results['real']])
               for name in ['LLL', 'LLH', 'LHL', 'LHH', 'HLL', 'HLH', 'HHL', 'HHH']}
    for method in ['Deepfakes', 'Face2Face']:
        if method in all_results:
            fake_sb = {name: np.mean([r['all_bands'].get(name, 0) for r in all_results[method]])
                       for name in real_sb}
            ratios = {name: fake_sb[name] / max(real_sb[name], 1e-10) for name in real_sb}
            best = max(ratios, key=ratios.get)
            print(f"   {method}: most discriminative sub-band = {best} ({ratios[best]:.2f}x)")
            
print(f"\n4. SBI PROXY QUALITY")
if sbi_results and real_source_results and ff_df_hll:
    sbi_med    = np.median(sbi_results)
    ff_df_med  = np.median(ff_df_hll)
    quality    = 'GOOD PROXY' if abs(sbi_med - ff_df_med) / max(ff_df_med, 1e-10) < 0.3 else 'POOR PROXY'
    print(f"   SBI HLL median:   {sbi_med:.6f}")
    print(f"   FF++ DFakes HLL:  {ff_df_med:.6f}")
    print(f"   Assessment:       {quality}")

print(f"\n5. V8.0 DESIGN RECOMMENDATIONS")
print("   Based on the data above:")
print("   [ ] Keep HLL if separation ratio > 1.1x for majority of methods")
print("   [ ] Add frame-difference branch if FrameDiff > HLL on any method")
print("   [ ] Use contrastive HLL loss (not absolute MSE(HLL, 0))")
print("   [ ] Train on REAL FF++ deepfakes, not just SBI, if SBI is poor proxy")
print("   [ ] Focus augmentation on border/boundary regions if heatmaps show boundary concentration")
print()
print("=" * 70)
print(f"📁 All plots saved to: {PLOTS_DIR}")
print("=" * 70)


In [ ]:
# ─── Save all numerical results to JSON for reference ─────────────────────────
def to_serializable(obj):
    if isinstance(obj, np.ndarray): return obj.tolist()
    if isinstance(obj, (np.float32, np.float64)): return float(obj)
    if isinstance(obj, (np.int32, np.int64)): return int(obj)
    return obj

summary = {
    'hll_energy_stats': {
        method: {
            'mean':   float(np.mean(vals)),
            'median': float(np.median(vals)),
            'std':    float(np.std(vals)),
            'count':  len(vals)
        }
        for method, vals in hll_vals.items()
    },
    'frame_diff_stats': {
        method: {
            'mean':   float(np.mean(vals)),
            'median': float(np.median(vals)),
        }
        for method, vals in fd_vals.items()
    },
    'sbi_proxy': {
        'sbi_median':          float(np.median(sbi_results)) if sbi_results else None,
        'real_source_median':  float(np.median(real_source_results)) if real_source_results else None,
        'ff_deepfakes_median': float(np.median(hll_vals.get('Deepfakes', [0])))
    }
}

out_path = OUTPUT_DIR / 'analysis_results.json'
with open(out_path, 'w') as f:
    json.dump(summary, f, indent=2, default=to_serializable)
print(f"✅ Numerical results saved → {out_path}")
print("\n🏁 Analysis complete! Review plots and report above before designing V8.0.")
